In [1]:
import pycolmap
from pathlib import Path
import shutil
import urllib.request
import zipfile
import io
import matplotlib.pyplot as plt
from PIL import Image

### Passo 1: Download e Preparação do Conjunto de Dados

Para este exemplo, usaremos o conjunto de dados "fox" do site oficial do COLMAP. O código abaixo irá baixá-lo e extraí-lo para a nossa pasta de imagens.

**IMPORTANTE: rodar a célula abaixo somente se for testar o código com o database de exemplo.**

In [ ]:
# Limpa execuções anteriores para garantir um começo limpo
if Path('colmap_project').exists():
    shutil.rmtree('colmap_project')

# Define os diretórios para o nosso projeto
project_path = Path('colmap_project')
image_path = project_path / 'images'
database_path = project_path / 'database.db'
output_path = project_path / 'sparse'

# Cria os diretórios necessários
project_path.mkdir(exist_ok=True)
image_path.mkdir(exist_ok=True)
output_path.mkdir(exist_ok=True)

print(f"Diretórios do projeto criados em: {project_path.resolve()}")

# URL do dataset "fox"
dataset_url = "https://github.com/colmap/colmap/raw/dev/data/datasets/fox.zip"

print("Baixando o conjunto de dados 'fox'...")
with urllib.request.urlopen(dataset_url) as response:
    data = response.read()

print("Extraindo imagens...")
with zipfile.ZipFile(io.BytesIO(data)) as z:
    # Extrai apenas os arquivos de imagem para o nosso diretório
    for member in z.infolist():
        if member.filename.startswith('images/') and member.filename.endswith(('.png', '.jpg', '.jpeg')):
            z.extract(member, project_path)

print("Download e extração concluídos.")

# Vamos visualizar algumas das imagens para confirmar
image_files = sorted(list(image_path.glob('*')))
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, img_file in enumerate(image_files[:4]):
    img = Image.open(img_file)
    axes[i].imshow(img)
    axes[i].set_title(img_file.name)
    axes[i].axis('off')
plt.show()

### Passo 1 (Alterado): Usando Imagens de uma Pasta Local

Esta etapa agora aponta para um diretório local chamado "imagens-t2/Imagens4".

Ação Necessária: Crie uma pasta chamada imagens-t2/Imagens4 no mesmo diretório onde este notebook está salvo e coloque suas imagens de objeto nela.

**IMPORTANTE: rodar a célula abaixo somente se for executar o código com as imagens capturadas para o projeto.**

In [ ]:
# Limpa execuções anteriores para garantir um começo limpo
if Path('colmap_project').exists():
    shutil.rmtree('colmap_project')

# Define os diretórios para os arquivos gerados pelo COLMAP
project_path = Path('colmap_project')
database_path = project_path / 'database.db'
output_path = project_path / 'sparse'

# Cria os diretórios para o projeto (exceto o de imagens, que será local)
project_path.mkdir(exist_ok=True)
output_path.mkdir(exist_ok=True)

print(f"Diretórios do projeto para a base de dados e saídas criados em: {project_path.resolve()}")

# Define o caminho para a pasta de imagens local
image_path = Path('imagens-t2/Imagens4')

# Verifica se a pasta existe e não está vazia
if not image_path.exists() or not any(image_path.iterdir()):
    print("="*80)
    print(f"ERRO: A pasta '{image_path}' não foi encontrada ou está vazia.")
    print("Por favor, crie esta pasta no mesmo diretório do notebook e adicione suas imagens.")
    print("="*80)
    # Lança um erro para parar a execução do notebook
    raise FileNotFoundError(f"A pasta de imagens '{image_path}' está faltando ou vazia.")
else:
    print(f"Usando imagens da pasta local: '{image_path.resolve()}'")

# Vamos visualizar algumas das imagens para confirmar
image_files = sorted(list(image_path.glob('*.[jJ][pP][gG]')) + list(image_path.glob('*.[pP][nN][gG]')))
print(f"Encontradas {len(image_files)} imagens.")

fig, axes = plt.subplots(1, min(4, len(image_files)), figsize=(16, 4))
if len(image_files) > 1:
    for i, img_file in enumerate(image_files[:4]):
        img = Image.open(img_file)
        axes[i].imshow(img)
        axes[i].set_title(img_file.name)
        axes[i].axis('off')
elif len(image_files) == 1:
    img = Image.open(image_files[0])
    axes.imshow(img)
    axes.set_title(image_files[0].name)
    axes.axis('off')
    
plt.show()

### Passo 2: Extração de Features (Keypoints)

Neste passo, o COLMAP analisará cada imagem para encontrar pontos de interesse distintos (features). Pelo que foi pesquisado, o algoritmo usado é o SIFT.

Atenção: Se você tiver uma GPU NVIDIA com CUDA configurado, pode definir use_gpu=True para uma aceleração massiva.

In [ ]:
# Configurações para a extração de features
feature_options = pycolmap.SiftExtractionOptions()
feature_options.use_gpu = False # Mude para True se tiver uma GPU compatível

# Executa a extração
pycolmap.extract_features(database_path, image_path, sift_options=feature_options)

print("Extração de features concluída.")

### Passo 3: Correspondência de Features (Matching)

Agora, o COLMAP irá comparar as features extraídas entre todas as imagens para encontrar correspondências (matches). Pelo que foi pesquisado, o algoritmo utilizado é: Nearest Neighbor + Verificação Geométrica com RANSAC.

In [ ]:
# Configurações para a correspondência
matcher_options = pycolmap.SiftMatchingOptions()
matcher_options.use_gpu = False # Mude para True se tiver uma GPU compatível

# Executa a correspondência
pycolmap.match_features(database_path, sift_options=matcher_options)

print("Correspondência de features concluída.")

### Passo 4: Reconstrução Esparsa (Structure from Motion)

Este é o passo principal. Usando as correspondências de features, o pycolmap executará o mapeamento incremental (SfM) para estimar simultaneamente:

- A estrutura 3D da cena (na forma de uma nuvem de pontos esparsa).

- Os parâmetros intrínsecos e extrínsecos da câmera para cada imagem (ou seja, onde cada foto foi tirada).

In [ ]:
# Executa o mapeamento incremental (reconstrução esparsa)
reconstructions = pycolmap.incremental_mapping(database_path, image_path, output_path)

print("Reconstrução esparsa concluída.")

# Imprime um resumo do maior modelo reconstruído
if reconstructions and 0 in reconstructions:
    print("\nResumo da Reconstrução:")
    print(reconstructions[0].summary())
else:
    print("\nA reconstrução falhou. Verifique se as imagens têm sobreposição suficiente e boa qualidade.")

### Passo 5: Visualização dos Resultados

A melhor parte! Vamos carregar o modelo esparso que acabamos de criar e visualizá-lo interativamente. O pycolmap tem uma integração fantástica com a biblioteca pythreejs para fazer isso diretamente no Jupyter.

Você poderá usar o mouse para rotacionar, dar zoom e mover a cena 3D. Os troncos de pirâmide azuis representam as câmeras.

In [ ]:
from pycolmap.visualization import visualize_model

# Carrega a reconstrução a partir dos arquivos de saída
try:
    reconstruction = pycolmap.Reconstruction(output_path)
    
    # Visualiza o modelo 3D
    if reconstruction and not reconstruction.is_empty():
        print("Renderizando visualização 3D interativa...")
        visualize_model(reconstruction, image_dir=image_path, point_size=3.0)
    else:
        print("Nenhum modelo reconstruído para visualizar.")

except ValueError as e:
    print(f"Erro ao carregar o modelo para visualização: {e}")
    print("Isso geralmente acontece se a reconstrução falhou na etapa anterior.")

### (Opcional) Passo 6: Reconstrução Densa (Multi-View Stereo)

O resultado acima é uma "nuvem de pontos esparsa". Para obter um modelo 3D mais detalhado, podemos realizar a reconstrução densa. Isso gera um mapa de profundidade para cada imagem e depois os funde em uma nuvem de pontos densa.

**AVISO: Este processo é computacionalmente muito mais intensivo.**

In [ ]:
# Verifica se a reconstrução esparsa existe antes de prosseguir
if 'reconstruction' in locals() and reconstruction and not reconstruction.is_empty():
    # Define o diretório para a reconstrução densa
    dense_path = project_path / 'dense'
    dense_path.mkdir(exist_ok=True)

    # 1. Desdistorcer as imagens
    pycolmap.undistort_images(dense_path, output_path, image_path)

    # 2. Executar o patch_match_stereo para criar mapas de profundidade
    pycolmap.patch_match_stereo(dense_path)

    # 3. Fundir os mapas de profundidade em uma nuvem de pontos densa
    pycolmap.stereo_fusion(dense_path / 'fused.ply', dense_path)

    print("\nReconstrução densa concluída!")
    print(f"O modelo 3D denso foi salvo em: {dense_path / 'fused.ply'}")
    print("Você pode abrir este arquivo .ply com um visualizador 3D como o MeshLab.")
else:
    print("\nPulando a reconstrução densa porque o modelo esparso não foi gerado com sucesso.")